In [1]:
import breeze.linalg._

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-9RG91OPS:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1669721950392)
SparkSession available as 'spark'


import breeze.linalg._


22/11/29 19:39:28 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


In [79]:
val x=Array(Array(1.0,2.0,3.0),Array(2.0,3.0,4.0),Array(3.0,4.0,5.0))
val y=Array(Array(2.0,3.0,4.0),Array(3.0,4.0,5.0),Array(4.0,5.0,6.0))  
val nr=x.length

x: Array[Array[Double]] = Array(Array(1.0, 2.0, 3.0), Array(2.0, 3.0, 4.0), Array(3.0, 4.0, 5.0))
y: Array[Array[Double]] = Array(Array(2.0, 3.0, 4.0), Array(3.0, 4.0, 5.0), Array(4.0, 5.0, 6.0))
nr: Int = 3


In [80]:
//关键：生成一个有坐标的结果矩阵，再把x，y的对应列放到这个结果Array的对应位置，再乘起来做sum则可。

In [81]:
var out=(0 until nr).toArray.map(f => {
    (0 until nr).toArray.map(s => (f,s))
})  //不能再用mat=了，因为数据类型不一样啊（scala对数据类型要求很严格啊，就是详细框架都不能错）
//所谓详细框架，是维数数组类型和其中的元素类型，如：Array[Array[Double]]

out: Array[Array[(Int, Int)]] = Array(Array((0,0), (0,1), (0,2)), Array((1,0), (1,1), (1,2)), Array((2,0), (2,1), (2,2)))


In [82]:
var mat_array=out.map(s => {
    s.map(f => {
        (DenseVector(x(f._1)),DenseVector(y(f._2)))
    })
})
//map只要框架不错就行，所谓详细框架，是维数数组类型，如：Array[Array[]]

mat_array: Array[Array[(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double])]] = Array(Array((DenseVector(1.0, 2.0, 3.0),DenseVector(2.0, 3.0, 4.0)), (DenseVector(1.0, 2.0, 3.0),DenseVector(3.0, 4.0, 5.0)), (DenseVector(1.0, 2.0, 3.0),DenseVector(4.0, 5.0, 6.0))), Array((DenseVector(2.0, 3.0, 4.0),DenseVector(2.0, 3.0, 4.0)), (DenseVector(2.0, 3.0, 4.0),DenseVector(3.0, 4.0, 5.0)), (DenseVector(2.0, 3.0, 4.0),DenseVector(4.0, 5.0, 6.0))), Array((DenseVector(3.0, 4.0, 5.0),DenseVector(2.0, 3.0, 4.0)), (DenseVector(3.0, 4.0, 5.0),DenseVector(3.0, 4.0, 5.0)), (DenseVector(3.0, 4.0, 5.0),DenseVector(4.0, 5.0, 6.0))))


In [83]:
var mat_rdd=sc.parallelize(mat_array)

mat_rdd: org.apache.spark.rdd.RDD[Array[(breeze.linalg.DenseVector[Double], breeze.linalg.DenseVector[Double])]] = ParallelCollectionRDD[7] at parallelize at <console>:28


In [84]:
//x*y.t
mat_rdd.map(s => {
    s.map(f => {
        f._1.t*f._2
    })
}).collect()

res53: Array[Array[Double]] = Array(Array(20.0, 26.0, 32.0), Array(29.0, 38.0, 47.0), Array(38.0, 50.0, 62.0))


In [85]:
//x+y
mat_rdd.map(s => {
    s.map(f => {
        f._1+f._2
    })
}).collect()

res54: Array[Array[breeze.linalg.DenseVector[Double]]] = Array(Array(DenseVector(3.0, 5.0, 7.0), DenseVector(4.0, 6.0, 8.0), DenseVector(5.0, 7.0, 9.0)), Array(DenseVector(4.0, 6.0, 8.0), DenseVector(5.0, 7.0, 9.0), DenseVector(6.0, 8.0, 10.0)), Array(DenseVector(5.0, 7.0, 9.0), DenseVector(6.0, 8.0, 10.0), DenseVector(7.0, 9.0, 11.0)))
